In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from scipy import spatial
import csv
import numpy as np
import math

In [2]:
base_dir = './Quora_question_pair_partition/'
train_dir = base_dir + 'train.tsv'
dev_dir = base_dir + 'dev.tsv'
test_dir = base_dir + 'test.tsv'
word_dir = base_dir + 'wordvec.txt'
words_vec = {}

In [3]:
def loadWordDic():
    with open(word_dir, 'r') as words:
        tweets_reader = csv.reader(words, delimiter=' ')
        cnt = 0
        for row in tweets_reader:
            words_vec[row[0]] = np.array([float(x) for x in row[1:]]).reshape((1,300))
            cnt += 1
            if cnt%5000 == 0:
                print(cnt)
loadWordDic()

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000


In [4]:
words1 = ['What', 'is', 'your', 'review', 'of', 'Hidden', 'Figures', '-LRB-', '2016', 'movie', '-RRB-', '?']
words2 = ['What', 'are', 'your', 'impressions', 'of', 'Hidden', 'Figures', '-LRB-', '2017', 'movie', '-RRB-', '?']
v1 = np.zeros((1,300))
cnt1 = 0
v2 = np.zeros((1,300))
cnt2 = 0
for word in words1:
    if word.lower() in words_vec:
        cnt1 += 1
        v1 += words_vec[word.lower()]

for word in words2:
    if word.lower() in words_vec:
        cnt2 += 1
        v2 += words_vec[word.lower()]

print ((v1 / cnt1 - v2  / cnt2).sum())

-0.5415105483333338


In [5]:
words1 = ['What', 'is', 'your', 'review', 'of', 'Hidden', 'Figures', '-LRB-', '2016', 'movie', '-RRB-', '?']
words2 = ['What', 'are', 'your', 'impressions', 'of', 'Hidden', 'Figures', '-LRB-', '2017', 'movie', '-RRB-', '?']

same_count = 0
for word in words1:
    if word in words2:
        same_count += 1

print(same_count / len(words1))
print(same_count / len(words2))
        

0.75
0.75


In [6]:
words1 = ['What', 'is', 'your', 'review', 'of', 'Hidden', 'Figures', '-LRB-', '2016', 'movie', '-RRB-', '?']
words2 = ['What', 'are', 'your', 'impressions', 'of', 'Hidden', 'Figures', '-LRB-', '2017', 'movie', '-RRB-', '?']

max_similarity = []

for word1 in words1:
    temp = 0
    for word2 in words2:
        if(word1.lower() in words_vec and word2.lower() in words_vec):
            value = 1 - spatial.distance.cosine(words_vec[word1.lower()], words_vec[word2.lower()])
            #print(word1 + ' ' + word2 + ' ' + str(value))
            if(value > temp):
                temp = value
    max_similarity.append(temp)

print(max_similarity)
print(np.mean(max_similarity))
print(np.median(max_similarity))
max_similarity.sort(reverse=True)
print(np.mean(max_similarity[:(len(max_similarity)//2)]))

[1.0, 0.5817388122944436, 1.0, 0.35610330124787437, 1.0, 1.0, 1.0, 1.0, 0.8642884698482242, 1.0, 1.0, 1.0]
0.9001775486158786
1.0
1.0


In [7]:
# read data
def loadData():
    train_df = pd.read_csv(train_dir,sep='\t', header=None, names=['label','q1','q2','id'],nrows =10000)
    train_X, train_y = train_df[['q1','q2']], train_df['label']
    dev_df = pd.read_csv(dev_dir,sep='\t', header=None, names=['label','q1','q2','id'])
    dev_X, dev_y = dev_df[['q1','q2']], dev_df['label']
    test_df = pd.read_csv(test_dir,sep='\t', header=None, names=['label','q1','q2','id'])
    test_X, test_y = test_df[['q1','q2']], test_df['label']
    return train_X, train_y, dev_X, dev_y, test_X, test_y

In [8]:
#train_X, train_y, dev_X, dev_y, test_X, test_y = loadData()
#temp = test_X.apply(lambda x: [1, 2], axis=1)
#temp.columns = ["new1","new2"]
#print(temp)

In [9]:
def fre1(row):
    words1 = row['q1'].split(' ')
    words2 = row['q2'].split(' ')
    same_count = 0
    for word in words1:
        if word in words2:
            same_count += 1

    return (same_count / len(words1))

def fre2(row):
    words1 = row['q1'].split(' ')
    words2 = row['q2'].split(' ')
    same_count = 0
    for word in words1:
        if word in words2:
            same_count += 1

    return (same_count / len(words2))

def dis_mean1(row):
    words1 = row['q1'].split(' ')
    words2 = row['q2'].split(' ')

    max_similarity = []
    for word1 in words1:
        temp = 0
        for word2 in words2:
            if (word1.lower() in words_vec and word2.lower() in words_vec):
                value = 1 - spatial.distance.cosine(words_vec[word1.lower()], words_vec[word2.lower()])
                # print(word1 + ' ' + word2 + ' ' + str(value))
                if (value > temp):
                    temp = value
        max_similarity.append(temp)

    value = np.mean(max_similarity)
    if(math.isnan(value)):
        print("nan occur")
        return 0.5

    return value


def dis_median1(row):
    words1 = row['q1'].split(' ')
    words2 = row['q2'].split(' ')

    max_similarity = []
    for word1 in words1:
        temp = 0
        for word2 in words2:
            if (word1.lower() in words_vec and word2.lower() in words_vec):
                value = 1 - spatial.distance.cosine(words_vec[word1.lower()], words_vec[word2.lower()])
                # print(word1 + ' ' + word2 + ' ' + str(value))
                if (value > temp):
                    temp = value
        max_similarity.append(temp)

    value = np.median(max_similarity)
    if(math.isnan(value)):
        print("nan occur")
        return 0.5

    return value

def dis_core_mean1(row):
    words1 = row['q1'].split(' ')
    words2 = row['q2'].split(' ')

    max_similarity = []
    for word1 in words1:
        temp = 0
        for word2 in words2:
            if (word1.lower() in words_vec and word2.lower() in words_vec):
                value = 1 - spatial.distance.cosine(words_vec[word1.lower()], words_vec[word2.lower()])
                # print(word1 + ' ' + word2 + ' ' + str(value))
                if (value > temp):
                    temp = value
        max_similarity.append(temp)

    max_similarity.sort(reverse=True)
    value = np.mean(max_similarity[:(len(max_similarity) // 2)])

    if(math.isnan(value)):
        print("nan occur")
        return 0.5

    return value

def dis_mean2(row):
    words1 = row['q2'].split(' ')
    words2 = row['q1'].split(' ')

    max_similarity = []
    for word1 in words1:
        temp = 0
        for word2 in words2:
            if (word1.lower() in words_vec and word2.lower() in words_vec):
                value = 1 - spatial.distance.cosine(words_vec[word1.lower()], words_vec[word2.lower()])
                # print(word1 + ' ' + word2 + ' ' + str(value))
                if (value > temp):
                    temp = value
        max_similarity.append(temp)

    value = np.mean(max_similarity)
    if(math.isnan(value)):
        print("nan occur")
        return 0.5

    return value


def dis_median2(row):
    words1 = row['q2'].split(' ')
    words2 = row['q1'].split(' ')

    max_similarity = []
    for word1 in words1:
        temp = 0
        for word2 in words2:
            if (word1.lower() in words_vec and word2.lower() in words_vec):
                value = 1 - spatial.distance.cosine(words_vec[word1.lower()], words_vec[word2.lower()])
                # print(word1 + ' ' + word2 + ' ' + str(value))
                if (value > temp):
                    temp = value
        max_similarity.append(temp)

    value = np.median(max_similarity)
    if(math.isnan(value)):
        print("nan occur")
        return 0.5

    return value

def dis_core_mean2(row):
    words1 = row['q2'].split(' ')
    words2 = row['q1'].split(' ')

    max_similarity = []
    for word1 in words1:
        temp = 0
        for word2 in words2:
            if (word1.lower() in words_vec and word2.lower() in words_vec):
                value = 1 - spatial.distance.cosine(words_vec[word1.lower()], words_vec[word2.lower()])
                # print(word1 + ' ' + word2 + ' ' + str(value))
                if (value > temp):
                    temp = value
        max_similarity.append(temp)

    max_similarity.sort(reverse=True)
    value = np.mean(max_similarity[:(len(max_similarity) // 2)])

    if (math.isnan(value)):
        print("nan occur")
        return 0.5

    return value

def rate_len(row):
    words1 = row['q1'].split(' ')
    words2 = row['q2'].split(' ')
    len1 = len(words1)
    len2 = len(words2)

    rate = len1/len2
    if(rate > 1):
        rate = 1/rate

    return rate

In [10]:
def generate_features(df):
    feature_names = []

    # number of words
    df['cnt'] = df.apply(lambda row: abs(len(row['q1'].split(' ')) - len(row['q2'].split(' '))), axis=1)
    feature_names.append('cnt')

    # distance between sentences
    #df['dis'] = df.apply(dis, axis=1)
    #feature_names.append('dis')

    # magic feature: question frequency

    # same word frequency
    df['fre1'] = df.apply(fre1, axis=1)
    feature_names.append('fre1')
    df['fre2'] = df.apply(fre2, axis=1)
    feature_names.append('fre2')

    ''' try to add more than one columns one time but failed
    temp = df.apply(fre3, axis=1)
    temp.columns = ['fre1', 'fre2']
    df.join(temp.to_frame())
    df[['q1','q2']] = df.apply(fre3, axis=1)
    feature_names.append('q1')
    feature_names.append('q2')
    '''

    # cosine based distance statistics
    df['dis_mean1'] = df.apply(dis_mean1, axis=1)
    feature_names.append('dis_mean1')
    #df['dis_core_mean1'] = df.apply(dis_core_mean1, axis=1)
    #feature_names.append('dis_core_mean1')
    df['dis_median1'] = df.apply(dis_median1, axis=1)
    feature_names.append('dis_median1')
    df['dis_mean2'] = df.apply(dis_mean2, axis=1)
    feature_names.append('dis_mean2')
    #df['dis_core_mean2'] = df.apply(dis_core_mean2, axis=1)
    #feature_names.append('dis_core_mean2')
    df['dis_median2'] = df.apply(dis_median2, axis=1)
    feature_names.append('dis_median2')

    # relative length rate
    df['rate_len'] = df.apply(rate_len, axis=1)
    feature_names.append('rate_len')

    print("Finish generating features")

    return feature_names, df

In [17]:
feature_names2, df2 = generate_features(test_X)

Finish generating features


In [12]:
train_X, train_y, dev_X, dev_y, test_X, test_y = loadData()
print("Finish loading data")
feature_names, df = generate_features(train_X)

Finish loading data
Finish generating features


In [39]:
logreg = linear_model.LogisticRegression(penalty='l2',C=1e7)

logreg.fit(df[feature_names], train_y)

acc = logreg.score(df[feature_names], train_y)

print("Accuracy on the training set:", acc)

acc = logreg.score(df2[feature_names2], test_y)

print("Accuracy on the testing set:", acc)

Accuracy on the training set: 0.6998
Accuracy on the testing set: 0.7015
